### contexto:

la columna 'address' conntiene info importante sobre como tratar los jsons de estados. ya que contiene informacion de direccion y ciudades.

si se normaliza esta columna en una tabla aparte que contenga: la ciudad la direccion y codigo postal, mas la clave unica gmap_id, podemos agrupar por estados con sus ciudades.

con un eda sobre las categorias de los negocios podemos decidir que estados usar y cuales no, basandonos en datos y no en suposiciones.

informacion: 

aqui solo hare codigo, no voy a documentar, si tienen dudas se lo explico en una reunion que hagamos.

In [2]:
import pandas as pd

sitios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\google_sitios.parquet', engine='pyarrow')
sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2998428 entries, 0 to 3025010
Data columns (total 18 columns):
 #   Column          Dtype  
---  ------          -----  
 0   name            object 
 1   address         object 
 2   gmap_id         object 
 3   description     object 
 4   latitude        float64
 5   longitude       float64
 6   avg_rating      float64
 7   num_of_reviews  int64  
 8   price           object 
 9   state           object 
 10  url             object 
 11  monday          object 
 12  tuesday         object 
 13  wednesday       object 
 14  thursday        object 
 15  friday          object 
 16  saturday        object 
 17  sunday          object 
dtypes: float64(3), int64(1), object(14)
memory usage: 434.6+ MB


In [4]:
address = sitios[['gmap_id','address', 'name']]
address.head()

,gmap_id,address,name
0,0x88f16e41928ff687:0x883dad4fd048e8f8,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",Porter Pharmacy
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",City Textile
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",San Soo Dang
3,0x80c2c89923b27a41:0x32041559418d447,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",Nova Fabrics
4,0x80c2c632f933b073:0xc31785961fe826a6,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",Nobel Textile Co


In [15]:
# Separar la columna 'address' en nuevas columnas
address_split = address['address'].str.split(',', n=3, expand=True)

# Cambiar el nombre de las columnas
address_split.columns = ['nombre', 'direccion', 'ciudad', 'cod.postal']

In [16]:
address_split.head()

,nombre,direccion,ciudad,cod.postal
0,Porter Pharmacy,129 N Second St,Cochran,GA 31014
1,City Textile,3001 E Pico Blvd,Los Angeles,CA 90023
2,San Soo Dang,761 S Vermont Ave,Los Angeles,CA 90005
3,Nova Fabrics,2200 E 11th St,Los Angeles,CA 90021
4,Nobel Textile Co,719 E 9th St,Los Angeles,CA 90021


In [17]:
address = address[['gmap_id']].join(address_split)
address.head()

,gmap_id,nombre,direccion,ciudad,cod.postal
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Porter Pharmacy,129 N Second St,Cochran,GA 31014
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,City Textile,3001 E Pico Blvd,Los Angeles,CA 90023
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,761 S Vermont Ave,Los Angeles,CA 90005
3,0x80c2c89923b27a41:0x32041559418d447,Nova Fabrics,2200 E 11th St,Los Angeles,CA 90021
4,0x80c2c632f933b073:0xc31785961fe826a6,Nobel Textile Co,719 E 9th St,Los Angeles,CA 90021


In [20]:
# Lista unica de todas las ciudades. contiene errores, hay que ver como mejorar esa normarlizacion.
ciudades = address['ciudad'].unique().tolist()
ciudades

[' Cochran',
 ' Los Angeles',
 ' Commerce',
 ' La Mirada',
 ' Waukee',
 ' 85 NE Dartmoor Dr',
 None,
 ' Pleasant Hill',
 ' Mammoth Lakes',
 ' Miami',
 ' Fair Oaks',
 ' Roseville',
 ' McClellan Park',
 ' Carmichael',
 ' Sacramento',
 ' Livingston',
 ' Shreveport',
 ' Saline',
 ' 818 Pierremont Rd',
 ' Little Elm',
 ' Frisco',
 ' Garland',
 ' Irving',
 ' Carrollton',
 ' Summersville',
 ' KS 67554',
 ' Gassaway',
 ' CA 92243',
 ' Pinellas Park',
 ' MI 48227',
 ' Westland',
 ' Southfield',
 ' San Diego',
 ' Hanford',
 ' Auburn',
 ' Penryn',
 ' 6939 Sunrise Blvd #250',
 ' Elkhart',
 ' Seminole',
 ' Easton',
 ' Brooklyn',
 ' Queens',
 ' New York',
 ' Medina',
 ' Tannersville',
 ' Colfax',
 ' Woodland',
 ' Vista',
 ' Philadelphia',
 ' Dodgeville',
 ' Montfort',
 ' Platteville',
 ' Highland',
 ' Cassville',
 ' Madison',
 ' WA 98665',
 ' Vancouver',
 ' Garden Grove',
 ' 17901 Von Karman Ave Suite 600',
 ' Irvine',
 ' Aliso Viejo',
 ' 723 E Green St',
 ' Deltona',
 ' Indianapolis',
 ' La Jolla',

In [27]:
print('primera normalizacion contiene ',len(ciudades),' ciudades unicas')

primera normalizacion contiene  219972  ciudades unicas


In [23]:
# Verificar que no encuentra porque tiene espacios, esto llama a que hay que normalizar quitando espacios iniciales, mayusculas/minusculas, quitar puntos.
fila = address.loc[address['ciudad'] == '85 NE Dartmoor Dr']
fila

,gmap_id,nombre,direccion,ciudad,cod.postal


In [24]:
fila = address.loc[address['ciudad'] == ' 85 NE Dartmoor Dr']
fila

,gmap_id,nombre,direccion,ciudad,cod.postal
8,0x87ec235c54d25b31:0x3b75fb5facc602f,Sweet Rewards Gluten Free Bakery,LLC,85 NE Dartmoor Dr,"Waukee, IA 50263"


bueno hasta aca por el momento, si continuan el analisis, hagan un pull y un push al terminar para no romper todo.

ideas: la tabla addres solo tendria que tener gmap_id, estado_id, ciudad_id 

de ahi se haria una forma normal 3fn donde para estado_id estarian todas las ciudades_id. 

y para ciudades_id todos los nombres, todas las direcciones de esa ciudad, agregando tambien en el futuro latitud y longitud. 

complementar ideas si se ocurren 